I executed the command lines below on WSL (Windows Subsystem for Linux) because I have a Dell laptop. For this project, I created a conda environment named JK_tophat and installed neccesary tools. To install conda (and Jupyter Notebook so you can save your code), ask chatGPT for help. To execute each code box, hit Shift+Enter.

In [ ]:
#Because tophat (read alignment tool) requires python version <3
conda create -n JK_tophat python=2.7
conda activate JK_tophat # now you change your environment from (base) to (JK_tophat)
conda install samtools=1.2 bowtie2=2.2.5 tophat=2.0.14 cufflinks=2.2.1
# to double check tools are installed
samtools --version && echo && bowtie2 --version && echo && tophat --version echo && cufflinks --version
# navigate to your folder/directory
cd /mnt/c/Users/lammu/OneDrive/Desktop/your_folder

In [2]:
# This step is only neccesary for me (executing WSL commands from Jupyter Bash Notebook)
source ~/miniconda3/etc/profile.d/conda.sh
conda activate JK_tophat
# To check that JK_tophat is activated
conda info --envs # this lists all conda environments I've created
# I should see a * in the line of JK_tophat


# conda environments:
#
base                   /home/lam/miniconda3
JK                     /home/lam/miniconda3/envs/JK
JK_tophat            * /home/lam/miniconda3/envs/JK_tophat
centrifuge             /home/lam/miniconda3/envs/centrifuge
kb                     /home/lam/miniconda3/envs/kb
rnaseq                 /home/lam/miniconda3/envs/rnaseq
rnaseq_py38            /home/lam/miniconda3/envs/rnaseq_py38
sourmash               /home/lam/miniconda3/envs/sourmash



# Objective: 
Determine genes that are differentially expressed at different stages in the development of Arabidopsis thaliana shoot apical meristem. You collected samples at day 8 and day 16 (files “Day8.fastq” and “Day16.fastq”).

The reference genome is “athal_chr.fa” and the reference gene annotations are in “athal_genes.gtf”. Use default parameters unless otherwise specified.

In [3]:
# These are the files we need, you can download them to your folder. Let view them in my folder:
ls

Day16.fastq  Day8.fastq  WSL_commands.ipynb  athal_chr.fa  athal_genes.gtf


# Steps we will do:
1. Index reference genome file using bowtie2-build
2. Align RNA-seq reads (fastq) with indexed reference genome using tophat. (fastq -> BAM)
3. Assemble aligned reads into genes and transcripts using cufflinks. (BAM -> transcripts.gtf)
4. Use cuffcompare to compare the assembled transcripts against a set of reference gene annotations, exon-by-exon, to determine which genes and transcripts in the sample are known, and which ones are likely novel.
5. Merge transcripts from 2 experimental conditions (Day 8 and Day 16) using cuffmerge. (gtf -> gtf)
6. Differential Expression with cuffdiff to compare expression levels between Day8 and Day16: run cuffdiff with the ‘merged.gtf’ file as reference annotation, taking as input the two alignment files. (BAM files generated by tophat -> diff)

## Step 1: Index reference genome file “athal_chr.fa” with bowtie2-build

In [5]:
# Create a directory (dir) name athal. Copy athal_chr.fa to this dir and rename it to athal.fa
mkdir athal # now your_folder has a new folder named "athal"
cp athal_chr.fa athal/athal.fa
# go to athal folder
cd athal
# Create a bowtie index of the genome using bowtie2-build, with the prefix ‘athal’. This generates 6 .bt2 files into the athal directory we just created
bowtie2-build athal.fa athal

Settings:
  Output files: "athal.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  athal.fa
Building a SMALL index
Reading reference sizes
  Time reading reference sizes: 00:00:00
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:00
bmax according to bmaxDivN setting: 125000
Using parameters --bmax 93750 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these parameters: --bmax 93750 --dcv 1024
Constructing suffix-array element generato

In [8]:
# 6 indexes files (.bt2) generated by bowtie2
ls

athal.1.bt2  athal.3.bt2  athal.fa         athal.rev.2.bt2
athal.2.bt2  athal.4.bt2  athal.rev.1.bt2


In [9]:
# go up 1 directory to your_folder
cd ..
ls

Day16.fastq  WSL_commands.ipynb  athal_chr.fa
Day8.fastq   athal               athal_genes.gtf


## Step 2: Align the RNA-seq data with indexed genome using TopHat

-p 8: Use 8 threads (you can adjust this depending on the number of cores you have available).

-o Tophat/Day8: the output directories/folders where the results for each sample will be saved.

athal/athal: the 1st athal is the folder, the 2nd athal is the prefix for the Bowtie2 index we just created.

Day8.fastq is the RNA-seq read to be aligned

In [12]:
mkdir Tophat
mkdir Tophat/Day8
mkdir Tophat/Day16
tophat -p 8 -o Tophat/Day8 athal/athal Day8.fastq
tophat -p 8 -o Tophat/Day16 athal/athal Day16.fastq


[2025-04-22 16:17:21] Beginning TopHat run (v2.0.14)
-----------------------------------------------
[2025-04-22 16:17:21] Checking for Bowtie
		  Bowtie version:	 2.2.5.0
[2025-04-22 16:17:21] Checking for Bowtie index files (genome)..
[2025-04-22 16:17:21] Checking for reference FASTA file
[2025-04-22 16:17:21] Generating SAM header for athal/athal
[2025-04-22 16:17:21] Preparing reads
	 left reads: min. length=50, max. length=50, 63573 kept reads (0 discarded)
[2025-04-22 16:17:44] Mapping left_kept_reads to genome athal with Bowtie2 
[2025-04-22 16:17:47] Mapping left_kept_reads_seg1 to genome athal with Bowtie2 (1/2)
[2025-04-22 16:17:48] Mapping left_kept_reads_seg2 to genome athal with Bowtie2 (2/2)
[2025-04-22 16:17:49] Searching for junctions via segment mapping
	Coverage-search algorithm is turned on, making this step very slow
	Please try running TopHat again with the option (--no-coverage-search) if this step takes too much time or memory.
[2025-04-22 16:17:50] Retrieving 

### Question 1: How many alignments were produced for the ‘Day8’  RNA-seq data set? 
Info is in accepted_hits.bam

In [14]:
# To print Field Descriptions and view the first 2 alignments in BAM file
echo -e "QNAME\tFLAG\tRNAME\tPOS\tMAPQ\tCIGAR\tRNEXT\tPNEXT\tTLEN\tSEQ\tQUAL" # print out Field Descriptions manually
samtools view Tophat/Day8/accepted_hits.bam | head -n 2

QNAME	FLAG	RNAME	POS	MAPQ	CIGAR	RNEXT	PNEXT	TLEN	SEQ	QUAL
SRR1660397.18502384	16	4	103	50	3M1I46M	*	0	0	CCATTCAGGACATTCTGGTGGTGGACTCGGAGGCATGATAGCAGGTGCAG	JIIHGJJIGHJIIIIIJJIGHCIGFIIJIJJJJJJJJHHHHHFFFFFCC@	AS:i:-8	XN:i:0	XM:i:0	XO:i:1	XG:i:1	NM:i:1	MD:Z:49	YT:Z:UU	NH:i:1
SRR1660397.9998980	0	4	105	50	50M	*	0	0	TTCAGGACATTCTGGTGGTGGACTCGGAGGCATGATAGCAGGTGCAGCTG	@@CFFEDDFFHHHBG<CF<CFGHDHIIFGB=EHBDBDA?DE9DGIEFI@G	AS:i:-5	XN:i:0	XM:i:1	XO:i:0	XG:i:0	NM:i:1	MD:Z:0A49	YT:Z:UU	NH:i:1


In [15]:
# To count alingments in BAM file
samtools view -c Tophat/Day8/accepted_hits.bam

63845


### Q2: How many reads were mapped in ‘Day8’ RNA-seq data set?
Info is in align_summary.txt

In [16]:
# Display the content of align_summary.txt in the terminal
cat Tophat/Day8/align_summary.txt
# the number of mapped reads is 63489

Reads:
          Input     :     63573
           Mapped   :     63489 (99.9% of input)
            of these:       356 ( 0.6%) have multiple alignments (0 have >20)
99.9% overall read mapping rate.


### Q3: How many spliced alignments were reported for ‘Day8’ RNA-seq data set?
A spliced alignment would be marked with ‘N’ in the CIGAR field (column 6) of accepted_hits.bam

In [18]:
samtools view Tophat/Day8/accepted_hits.bam | cut -f6 | grep -c 'N'
# to view an example spliced alignment
samtools view Tophat/Day8/accepted_hits.bam | awk '$6 ~ /N/' | head -n 1

8596
SRR1660397.1102913	0	4	1266	3	49M579N1M	*	0	0	CACACGCTTTTTCATTTCAATCTTCTTCTTCTACTCTTAAGTATCTCAGG	@@@DDDADFFHGDHGIIGIIJGIIHIIJIJGCABHHIIGIIJIJJJIGEG	AS:i:0	XN:i:0	XM:i:0	XO:i:0	XG:i:0	NM:i:0	MD:Z:50	YT:Z:UU	XS:A:+	NH:i:2	HI:i:1


## Step 3: assemble the aligned RNA-seq reads into genes and transcripts using cufflinks. 
Use the labels ‘Day8’ and ‘Day16’, respectively, when creating identifiers.

In [19]:
mkdir Cufflinks
mkdir Cufflinks/Day8
mkdir Cufflinks/Day16
cufflinks -o Cufflinks/Day8 -L Day8 Tophat/Day8/accepted_hits.bam
cufflinks -o Cufflinks/Day16 -L Day16 Tophat/Day16/accepted_hits.bam

[17:06:59] Inspecting reads and determining fragment length distribution.
> Processed 321 loci.                          [*************************] 100%
> Map Properties:
>	Normalized Map Mass: 63489.00
>	Raw Map Mass: 63489.00
>	Fragment Length Distribution: Truncated Gaussian (default)
>	              Default Mean: 200
>	           Default Std Dev: 80
[17:07:00] Assembling transcripts and estimating abundances.
> Processed 327 loci.                          [*************************] 100%
[17:07:06] Inspecting reads and determining fragment length distribution.
> Processed 139 loci.                          [*************************] 100%
> Map Properties:
>	Normalized Map Mass: 57951.00
>	Raw Map Mass: 57951.00
>	Fragment Length Distribution: Truncated Gaussian (default)
>	              Default Mean: 200
>	           Default Std Dev: 80
[17:07:08] Assembling transcripts and estimating abundances.
> Processed 161 loci.                          [*************************] 100%


### Q4: How many genes were generated by cufflinks for Day8?
cufflinks generates ‘transcripts.gtf’ containing the assembled transcripts, as well as files ‘*.fpkm_tracking’ containing expression (FPKM) estimates for genes and transcripts.

In [21]:
# To view the first line of GTF file
echo -e "chrom\tsource\tfeature\tstart\tend\tscore\tstrand\tframe\tattribute" # print field description
head -n 1 Cufflinks/Day8/transcripts.gtf
# extract column 9 which contain gene_ID | extract gene_ID, ie, "Day8.1"; | sort and eliminate duplicate | count how many gene_IDs/genes in transcripts.gtf file
cut -f9 Cufflinks/Day8/transcripts.gtf | cut -d ' ' -f2 | sort -u | wc -l

chrom	source	feature	start	end	score	strand	frame	attribute
4	Cufflinks	transcript	103	721	1000	.	.	gene_id "Day8.1"; transcript_id "Day8.1.1"; FPKM "120347.2406938536"; frac "1.000000"; conf_lo "116072.911057"; conf_hi "124621.570330"; cov "382.036298";
186


### Q5: How many transcripts were reported for Day8? 

In [22]:
# extract column 9 which contain transcript_ID | extract transcript_ID, ie, "Day8.1.1"; | sort and eliminate duplicate | count how many transcript_IDs/transcripts in transcripts.gtf file
cut -f9 Cufflinks/Day8/transcripts.gtf | cut -d ' ' -f4 | sort -u | wc -l

192


### Q6: How many single transcript genes were produced for Day8? 
Each gene can have one or more transcripts. We first create a listing of (column 1 as gene, column 2 as transcript) pairs by "cut -d ' ' -f2,4" and use it to determine the number of transcripts for each gene. For instance, a gene with a single transcript would appear only 1 time in column 1, a gene with 2 transcripts 2 times, etc. We calculate the number of occurrences in column 1 for each gene using ‘uniq -c’, and then select those that have count 1.

In [25]:
# visualize the outcome of each command
cut -f9 Cufflinks/Day8/transcripts.gtf | cut -d ' ' -f2,4 | sort -u | head -n 2
cut -f9 Cufflinks/Day8/transcripts.gtf | cut -d ' ' -f2,4 | sort -u | cut -d ' ' -f1 | sort | uniq -c | head -n 2
# count genes with only 1 transcript
cut -f9 Cufflinks/Day8/transcripts.gtf | cut -d ' ' -f2,4 | sort -u | cut -d ' ' -f1 | sort | uniq -c | grep -c " 1 "

"Day8.1"; "Day8.1.1";
"Day8.10"; "Day8.10.1";
      1 "Day8.1";
      1 "Day8.10";
180


### Q7: How many single-exon transcripts were in the Day8 set?
Each transcript is represented in the GTF file with one ‘transcript’ (column 3) line and one or several ‘exon’ (column 3) lines. Therefore, a single-exon transcript would appear listed on exactly 2 lines: 1 line for transcript, 1 line for exon. Note that these 2 lines have the same transcript_id in column 9.

In [27]:
# visualize a transcript with single-exon
cut -f9 Cufflinks/Day8/transcripts.gtf | cut -d ' ' -f4 | sort | uniq -c | grep " 2 " | head -n 1
# count transcript with only 1 exon
cut -f9 Cufflinks/Day8/transcripts.gtf | cut -d ' ' -f4 | sort | uniq -c | grep -c " 2 "
# count transcript wit multiple exon by command line
cut -f9 Cufflinks/Day8/transcripts.gtf | cut -d ' ' -f4 | sort | uniq -c | awk '$1 > 2' | wc -l

      2 "Day8.1.1";
119
73


## Step 4: cuffcompare 
cuffcompare compares the assembled transcripts against a set of reference gene annotations provided by the user, exon-by-exon, to determine which genes and transcripts in the sample are known, and which ones are likely novel. 
In the end, it assigns each predicted (cufflinks) transcript a ‘class’ code depending on how it relates to a reference transcript, for example: it is the same as a reference transcript (‘=’), it is only a portion of one (‘c’), a new splice variant of a reference gene (‘j’), etc. 
Run cuffcompare against the provided annotation (‘-r’) and with the option ‘-R’ to exclude from statistics genes that do not appear to be represented in the sample.
These will create the files ‘day*.combined.gtf’ combining the reference and predicted annotations, ‘day*.transcripts.gtf.tmap’ containing a mapping between the assembled transcripts and the reference genes and transcripts, as well as other derived files, in the corresponding directories.

In [28]:
cuffcompare # see how to use cuffcompare

cuffcompare v2.2.1 (4237)
-----------------------------
Usage:
cuffcompare [-r <reference_mrna.gtf>] [-R] [-T] [-V] [-s <seq_path>] 
    [-o <outprefix>] [-p <cprefix>] 
    {-i <input_gtf_list> | <input1.gtf> [<input2.gtf> .. <inputN.gtf>]}

 Cuffcompare provides classification, reference annotation mapping and various
 statistics for Cufflinks transfrags.
 Cuffcompare clusters and tracks transfrags across multiple samples, writing
 matching transcripts (intron chains) into <outprefix>.tracking, and a GTF
 file <outprefix>.combined.gtf containing a nonredundant set of transcripts 
 across all input files (with a single representative transfrag chosen
 for each clique of matching transfrags across samples).

Options:
-i provide a text file with a list of Cufflinks GTF files to process instead
   of expecting them as command line arguments (useful when a large number
   of GTF files should be processed)

-r a set of known mRNAs to use as a reference for assessing 
   the accuracy of mRN

: 1

In [30]:
mkdir -p cuffcompare/Day8 # -p prevent error message when cuffcompare/Day8 already exist
cuffcompare -r athal_genes.gtf -R -o cuffcompare/Day8/day8 Cufflinks/Day8/transcripts.gtf
# -o cuffcompare/Day8/day8 will generate cuffcompare/Day8/day8.combined.gtf and other files. Note that this command output day8.transcripts.gtf.tmap in Cufflinks/Day8/

In [31]:
mkdir -p cuffcompare/Day16
cuffcompare -r athal_genes.gtf -R -o cuffcompare/Day16/day16 Cufflinks/Day16/transcripts.gtf

### Q8: How many cufflinks transcripts fully reconstruct annotation transcripts in Day8?
transcripts fully reconstruct annotation transcripts is indicated by the ‘class’ code "=" in column 3.

In [33]:
# to view the first 2 lines of .tmap file
head -n 2 Cufflinks/Day8/day8.transcripts.gtf.tmap

ref_gene_id	ref_id	class_code	cuff_gene_id	cuff_id	FMI	FPKM	FPKM_conf_lo	FPKM_conf_hi	cov	len	major_iso_id	ref_match_len
AT4G19200	AT4G19200.1	=	Day8.1	Day8.1.1	100	120347.240694	116072.911057	124621.570330	382.036298	619	Day8.1.1	613


In [32]:
cut -f3 Cufflinks/Day8/day8.transcripts.gtf.tmap | sort | uniq -c 
# the answer is 16
# there are 133 cufflinks transcripts that are partial reconstructions of reference transcripts, indicated by class_code "c"
# there are 4 cufflinks transcripts that were formed in the introns of reference genes, indicated by class_code "i"
# there are 14 cufflinks transcripts that are novel splice variants of reference genes, indicated by class_code "j"


     16 =
    133 c
      1 class_code
      7 e
      4 i
     14 j
     15 o
      3 p


### Q9: How many splice variants does the gene AT4G20240 have in the Day8 sample? 

In [34]:
grep "AT4G20240" Cufflinks/Day8/day8.transcripts.gtf.tmap

AT4G20240	AT4G20240.1	j	Day8.159	Day8.159.1	100	6112.378433	5517.096482	6707.660384	19.403440	1294	Day8.159.1	1608
AT4G20240	AT4G20240.1	o	Day8.160	Day8.160.1	100	4289.867816	3045.166614	5534.569017	13.617971	380	Day8.160.1	1608


In [35]:
grep "AT4G20240" Cufflinks/Day8/day8.transcripts.gtf.tmap | wc -l # the answer is 2

2


# Step 5: Merge transcripts using cuffmerge

In [36]:
# First, prepare a text file (e.g., assemblies.txt) that lists the paths to both transcripts.gtf files:
echo Cufflinks/Day8/transcripts.gtf > assemblies.txt
echo Cufflinks/Day16/transcripts.gtf >> assemblies.txt

In [38]:
cuffmerge -g athal_genes.gtf assemblies.txt # this will create a directory ‘merged_asm’ containing the file ‘merged.gtf’.


[Tue Apr 22 18:40:13 2025] Beginning transcriptome assembly merge
-------------------------------------------

[Tue Apr 22 18:40:13 2025] Preparing output location ./merged_asm/
[Tue Apr 22 18:40:13 2025] Converting GTF files to SAM
[18:40:13] Loading reference annotation.
[18:40:13] Loading reference annotation.
[Tue Apr 22 18:40:13 2025] Quantitating transcripts
Command line:
cufflinks -o ./merged_asm/ -F 0.05 -g athal_genes.gtf -q --overhang-tolerance 200 --library-type=transfrags -A 0.0 --min-frags-per-transfrag 0 --no-5-extend -p 1 ./merged_asm/tmp/mergeSam_fileEZpXzc 
[bam_header_read] EOF marker is absent. The input is probably truncated.
[bam_header_read] invalid BAM binary header (this is not a BAM file).
File ./merged_asm/tmp/mergeSam_fileEZpXzc doesn't appear to be a valid BAM file, trying SAM...
[18:40:13] Loading reference annotation.
[18:40:13] Inspecting reads and determining fragment length distribution.
Processed 119 loci.                         
> Map Properties:
>	

### Q10: How many genes (loci) were reported in the merged.gtf file?

In [39]:
echo -e "chrom\tsource\tfeature\tstart\tend\tscore\tstrand\tframe\tattribute" # print field description
head -n 1 merged_asm/merged.gtf
cut -f9 merged_asm/merged.gtf | cut -d ' ' -f2 | sort -u | wc -l

chrom	source	feature	start	end	score	strand	frame	attribute
4	Cufflinks	exon	110	722	.	+	.	gene_id "XLOC_000001"; transcript_id "TCONS_00000001"; exon_number "1"; gene_name "AT4G19200"; oId "AT4G19200.1"; nearest_ref "AT4G19200.1"; class_code "="; tss_id "TSS1";
129


# Step 6: Differential Expression with cuffdiff to compare expression levels between Day8 and Day16
run cuffdiff with the ‘merged.gtf’ file as reference annotation, taking as input the two alignment files and directing the output to the directory Cuffdiff (‘-o’)

In [40]:
mkdir Cuffdiff
cuffdiff -o Cuffdiff merged_asm/merged.gtf Tophat/Day8/accepted_hits.bam Tophat/Day16/accepted_hits.bam
# This will create the file ‘gene_exp.diff’ containing test scores and results for the gene-level differential expression analysis, 
# also generate other ‘*.diff’ files, as well as tracking files for genes, transcripts, splicing, CDS, TSS, etc.

[18:47:19] Loading reference annotation.
[18:47:19] Inspecting maps and determining fragment length distributions.
[18:47:20] Modeling fragment count overdispersion.
> Map Properties:
>	Normalized Map Mass: 61354.33
>	Raw Map Mass: 62678.00
>	Fragment Length Distribution: Truncated Gaussian (default)
>	              Default Mean: 200
>	           Default Std Dev: 80
> Map Properties:
>	Normalized Map Mass: 61354.33
>	Raw Map Mass: 56432.50
>	Fragment Length Distribution: Truncated Gaussian (default)
>	              Default Mean: 200
>	           Default Std Dev: 80
[18:47:53] Calculating preliminary abundance estimates
[18:47:53] Testing for differential expression and regulation in locus.
> Processed 116 loci.                          [*************************] 100%
Performed 111 isoform-level transcription difference tests
Performed 83 tss-level transcription difference tests
Performed 66 gene-level transcription difference tests
Performed 0 CDS-level transcription difference tests


### Q11: How many genes total were included in the gene expression report from cuffdiff?
count the number of lines in the ‘gene_exp.diff’ file and subtract 1 (the header):

In [41]:
head -n 2 Cuffdiff/gene_exp.diff
wc -l Cuffdiff/gene_exp.diff

test_id	gene_id	gene	locus	sample_1	sample_2	status	value_1	value_2	log2(fold_change)	test_stat	p_value	q_value	significant
XLOC_000001	XLOC_000001	AT4G19200	4:109-722	q1	q2	OK	152925	11993.6	-3.67249	-2.98827	0.0111	0.14652	no
130 Cuffdiff/gene_exp.diff


### Q12: How many genes were detected as differentially expressed?
look at column 13 "significant"

In [42]:
grep -c yes Cuffdiff/gene_exp.diff

4


### Q13: How many transcripts were differentially expressed between the two samples?

In [44]:
grep -c yes Cuffdiff/isoform_exp.diff

5


In [45]:
# When done with your analysis, deactivate JK_tophit
conda deactivate

In [46]:
# To check that JK_tophat is deactivated
conda info --envs # this lists all conda environments I've created
# I should see a * in the line of base


# conda environments:
#
base                 * /home/lam/miniconda3
JK                     /home/lam/miniconda3/envs/JK
JK_tophat              /home/lam/miniconda3/envs/JK_tophat
centrifuge             /home/lam/miniconda3/envs/centrifuge
kb                     /home/lam/miniconda3/envs/kb
rnaseq                 /home/lam/miniconda3/envs/rnaseq
rnaseq_py38            /home/lam/miniconda3/envs/rnaseq_py38
sourmash               /home/lam/miniconda3/envs/sourmash



# Testing your knowledge
You can try answering the above questions for the ‘Day16’  RNA-seq data set.
Then try these additional questions:
### Q1: How many reads were uniquely aligned in ‘Day8’ RNA-seq data set?
### Q2: How many reads were left unmapped from ‘Day8’ RNA-seq data set?